## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-02-24-06 +0000,nypost,Ex-porn star Austin Wolf sentenced to 19 years...,https://nypost.com/2025/09/30/us-news/ex-porn-...
1,2025-10-01-02-21-55 +0000,nyt,A More Polarized Senate Drives Shutdown Gridlock,https://www.nytimes.com/2025/09/30/us/politics...
2,2025-10-01-02-18-40 +0000,bbc,"US cities should be military training grounds,...",https://www.bbc.com/news/articles/cvgq044n72po...
3,2025-10-01-02-15-44 +0000,bbc,Australia sunscreen scandal grows as more prod...,https://www.bbc.com/news/articles/c62qdje2ll4o...
4,2025-10-01-02-12-54 +0000,nypost,Gov. Spencer Cox reflects on Charlie Kirk’s as...,https://nypost.com/2025/09/30/us-news/gov-spen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,57
13,shutdown,24
107,government,21
55,new,17
236,deal,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,144
120,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,127
314,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,123
122,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,120
306,2025-09-30-08-01-11 +0000,bbc,US shutdown looms after Trump and Democrats en...,https://www.bbc.com/news/articles/c2kn3n315lko...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,144,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
186,70,2025-09-30-16-30-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
120,59,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
178,54,2025-09-30-17-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
248,48,2025-09-30-13-35-39 +0000,nyt,"With New U.S. Proposal to End Gaza War, a Rare...",https://www.nytimes.com/2025/09/30/world/middl...
35,36,2025-10-01-00-24-27 +0000,nypost,Michigan church attack victims include husband...,https://nypost.com/us-news/victims-of-michigan...
204,36,2025-09-30-15-50-00 +0000,bbc,Trump peace plan 'ignores interests of Palesti...,https://www.bbc.com/news/articles/cx2j97jldkmo...
18,35,2025-10-01-01-22-00 +0000,wsj,The White House withdrew the nomination of con...,https://www.wsj.com/politics/policy/white-hous...
84,35,2025-09-30-22-04-22 +0000,nypost,Ford CEO warns there’s not enough blue-collar ...,https://nypost.com/2025/09/30/business/ford-ce...
188,34,2025-09-30-16-21-56 +0000,bbc,South Africa's top diplomat in France found de...,https://www.bbc.com/news/articles/ce3y505yq4ko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
